## Data_P6_Holdout_Processing

+ **Objective:** Format holdout data to modeldf to prep for model evaluation
+ **Author:** Cynthia He

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import s3fs
import spacy

import nltk
from nltk.tokenize import word_tokenize
# nltk.download('stopwords') #<<< one time effort

### Load and Process Data

In [12]:
bucket = 's3://cortex-ni-wc-claims'
file = 'WC_DeEscl_Data_Holdout_wClstr_v2.csv'

df = pd.read_csv(f'{bucket}/{file}')
print('Data shape: ', df.shape)

Data shape:  (136131, 100)


**Comments:** Shape is consistent with the build data dimension. One less conlumn because "folds" indicator us not needed.

In [5]:
df.head(5)

Year  Claim_ID  Claim_Num Claim_System_Entry_Dte  Report_dte  Injury_Dte  \
0  2013   2518606  608A55795             2013-01-04  2013-01-02  2013-01-01   
1  2013   2533606  608A56368             2013-01-11  2013-01-10  2013-01-09   
2  2013   2561806  949A88879             2013-02-18  2013-02-17  2013-02-16   
3  2013   2569906  12CA05378             2013-03-13  2013-03-12  2012-04-23   
4  2013   2571606  648A72070             2013-03-18  2013-03-17  2013-03-08   

  Claim_Status_Cde Current_PMT_Cde Current_Controverted_I  \
0                F              03                    NaN   
1                F              M4                      N   
2                F              01                      N   
3                F              03                    NaN   
4                F             NaN                    NaN   

   current_total_incurred  ...  PrimNOI_km_6c  PrimNOI_km_12c  PrimNOI_pam_6c  \
0                     0.0  ...              1              10               6   
1                   359.0  ...              5               8               4   
2                   976.0  ...              5               8               4   
3                     0.0  ...              2               3               5   
4                     0.0  ...              1               4               6   

  PrimNOI_pam_7c PrimNOI_pam_9c  Occur_km_3c Occur_km_4c Occur_km_6c  \
0              7              8            1           1           4   
1              5              5            3           3           1   
2              5              5            3           3           1   
3              6              6            3           3           1   
4              7              9            1           1           5   

   Occur_pam_4c Occur_pam_9c  
0             4            9  
1             2            5  
2             2            5  
3             2            5  
4             4            6  

[5 rows x 100 columns]

In [7]:
# fill missing in the notes
accident_notes = df['Accident_Note_Text']
accident_notes= accident_notes.fillna('unknown')
injury_notes = df['Injury_Note_Text']
injury_notes = injury_notes.fillna('unknown')

In [8]:
accident_notes.head(10)

0    ASSOCIATE DEVIN MARKESTEIN APPROACHED ASDS IN ...
1    MEMBER STATED THAT WHEN SHE WAS CARRYING A SAL...
2    SERVER WAS PREBUSSING HER TABLES SHE WENT TO L...
3    WAS TRYING TO GET OFF THE UTILITY CARTPLACED H...
4    ASSOCIATE SCRAPED HAND WHILE SWEEPING IN OUTSI...
5    ASSOCIATES BOB AGEE HAD A SEZIUR AS HE FELL TO...
6    SCOTT WAS LOADING A BAG OF CONCRETE WHEN HE FE...
7    WHILE LOADING 80 LB CONCRETE INTO A CUSTOMERS ...
8    WHILE HE WAS THROWING TRASH TABLE FELL ON BACK...
9    WHILE WORKING IN THE KITCHEN A CONTAINER OF HO...
Name: Accident_Note_Text, dtype: object

In [9]:
injury_notes.head(10)

0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
5    unknown
6    unknown
7    unknown
8    unknown
9    unknown
Name: Injury_Note_Text, dtype: object

### Text Cleaning

In [15]:
from nltk.corpus import stopwords
from ast import literal_eval
import re

nltk.download('punkt')
nltk.download('wordnet')

# from autocorrect import spell

[nltk_data] Downloading package punkt to /mnt/N0312164/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /mnt/N0312164/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = stopwords.words('english')
keep = {'not'}
STOPWORDS = [e for e in STOPWORDS if e not in keep]
extra_stopwords = ['left', 'right', 'employee', 'emp', 'ee', 'associate', 'iw']
STOPWORDS = set(STOPWORDS + extra_stopwords)
# !!!may need to revise the stop words list later

def text_prepare(text):
    """
        input: a string
        
        return: cleaned string
    """
    text = text.lower() # lowercase text
    text = re.sub("n't", " not", text)
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    word_tokens = word_tokenize(text) 
    word_tokens = [w for w in word_tokens if not w in STOPWORDS] # delete stopwords from text
  
    stemmer = nltk.stem.WordNetLemmatizer()
    for i in range(len(word_tokens)):
        # stemming + spelling correction
        
#          word_tokens[i] = token.lemma_( word_tokens[i])
        word_tokens[i] = stemmer.lemmatize(stemmer.lemmatize(word_tokens[i]), 'v') #lemmatize both noun and verb
#         word_tokens[i] = stemmer.lemmatize(spell(word_tokens[i]))
        
    text = " ".join(word_tokens)
    return text

In [16]:
import time
start_time = time.time()
accident_cleaned= accident_notes.apply(text_prepare)
print("--- %s seconds ---" % (time.time() - start_time))

--- 26.31200122833252 seconds ---


In [17]:
start_time = time.time()
injury_cleaned = injury_notes.apply(text_prepare)
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.212343692779541 seconds ---


In [18]:
accident_cleaned[:10]

0    devin markestein approach asds train room say ...
1    member state carry salad bin fell knee hit upp...
2    server prebussing table go lift tray felt shar...
3    try get utility cartplaced foot grind cart rol...
4    scrap hand sweep outside garden scrape since b...
5    associate bob agee seziur fell grind hit head ...
6    scott load bag concrete felt disconfort lower ...
7    load 80 lb concrete customer truck robert stra...
8    throw trash table fell back leg metal part tab...
9    work kitchen container hot water knock onto fi...
Name: Accident_Note_Text, dtype: object

In [19]:
accident_cleaned.to_frame().describe()

Accident_Note_Text
count              136131
unique             131816
top               unknown
freq                 1226

In [20]:
injury_cleaned[:10]

0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
5    unknown
6    unknown
7    unknown
8    unknown
9    unknown
Name: Injury_Note_Text, dtype: object

In [21]:
injury_cleaned.to_frame().describe()

Injury_Note_Text
count            136131
unique            50977
top             unknown
freq              34406

In [22]:
df['Acc_Txt_Clned'] = accident_cleaned
df['Inj_Txt_Clned'] =  injury_cleaned

In [23]:
# sanity check
df.shape

(136131, 102)

In [24]:
# save cleaned text with the original data fields
filename = 'WC_DeEscl_Data_Holdout_wClstr_v2_txtcln.csv'
df.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

In [32]:
# check empty cell found in cleaned text
df[df['Acc_Txt_Clned']==""][['Accident_Note_Text']] 

Accident_Note_Text
13073            EMPLOYEE
29192            EMPLOYEE
30918               OTHER
61508       SAME AS ABOVE
106231      SAME AS ABOVE

In [33]:
df[df['Inj_Txt_Clned']==""][['Injury_Note_Text']] 

Injury_Note_Text
1535              OTHER
5139              OTHER
9414              OTHER
11883             OTHER
12344             OTHER
12802             OTHER
13098             OTHER
14923              SAME
15447             OTHER
15451             OTHER
15900             OTHER
16168             OTHER
17271             OTHER
17646             OTHER
19662             OTHER
22473             OTHER
25570             OTHER
26837             OTHER
27330             OTHER
28108             OTHER
28239            EE WAS
28477             OTHER
29602             OTHER
29748             OTHER
30849             OTHER
31995             OTHER
38844             OTHER
51700             OTHER
62578             OTHER
73946             RIGHT
91480             OTHER
94290     SAME AS ABOVE
97148             OTHER
134641   LEFT AND RIGHT

In [30]:
#print revious line if the note state "SAME AS ABOVE"
for i in range(df.shape[0]):
    if df.Accident_Note_Text[i] in {'SAME', 'AS ABOVE', 'SAME AS ABOVE'}:
        print(i)
        print(df[['Claim_ID', 'Claim_Num', 'Report_dte', 'Account_Name', 'Accident_Note_Text', 'Injury_Note_Text', 'Acc_Txt_Clned']].iloc[(i-1):(i+1), :])

61508
        Claim_ID  Claim_Num  Report_dte        Account_Name  \
61507  153061706  608C49011  2015-11-17  THE HOME DEPOT INC   
61508  153063006  390D11408  2015-11-20  DEAN FOODS COMPANY   

                        Accident_Note_Text  \
61507  SMALL PUNTURE IN RIGHT INDEX FINGER   
61508                        SAME AS ABOVE   

                                        Injury_Note_Text  \
61507                                           PUNCTURE   
61508  EMPLOYEE WAS PICKING UP CASESAND WRENCHED HIS ...   

                    Acc_Txt_Clned  
61507  small punture index finger  
61508                              
106231
         Claim_ID  Claim_Num  Report_dte                  Account_Name  \
106230  482642306  555C14480  2013-10-03  COSTCO WHOLESALE CORPORATION   
106231  482693406  80DC15641  2014-01-02         ELI LILLY AND COMPANY   

                                       Accident_Note_Text Injury_Note_Text  \
106230  EMPLOYEE WAS LIFTING A BOX INTO A SHOPPING CAR...           

In [29]:
for i in range(df.shape[0]):
    if df.Injury_Note_Text[i] in {'SAME', 'AS ABOVE', 'SAME AS ABOVE'}:
        print(i)
        print(df[['Claim_ID', 'Claim_Num', 'Report_dte', 'Account_Name', 'Accident_Note_Text', 'Injury_Note_Text', 'Inj_Txt_Clned']].iloc[(i-1):(i+1), :])

14923
        Claim_ID  Claim_Num  Report_dte              Account_Name  \
14922  992789606  648C04186  2014-10-07        THE HOME DEPOT INC   
14923  992790806  608C10772  2014-10-10  DARDEN RESTAURANTS, INC.   

                                      Accident_Note_Text Injury_Note_Text  
14922  ASSOCIATE CLAIMS THAT CUSTOMER POKED ASSOCIATE...  BRUISECONTUSION  
14923  NINO SAYS HE FIRST INJURED HIS LEFT WRIST WHIL...             SAME  
94290
        Claim_ID  Claim_Num  Report_dte              Account_Name  \
94289  783056006  949C84533  2015-11-04        THE HOME DEPOT INC   
94290  783056306  555C95933  2015-11-04  DARDEN RESTAURANTS, INC.   

                                      Accident_Note_Text    Injury_Note_Text  
94289  ASSOCIATE WAS PUSHING MICROWAVES ACROSS AISLE ...  HERNIAGROIN STRAIN  
94290  EMPLOYEE SHARED A STEAK WITH ANOTHER EMPLOYEE ...       SAME AS ABOVE  


**Comments:** For the claims have notes indicating "same as above", the order of the reported claims may have been shifted. There is not enough evidence suggesting those fields can be filled by the content from the previous line. They will be marked as missing.

In [40]:
# fill empty cell with 'unknown'
df.loc[df['Acc_Txt_Clned']=='', 'Acc_Txt_Clned'] = 'unknown'
df.loc[df['Inj_Txt_Clned']=='', 'Inj_Txt_Clned'] = 'unknown'

# re-ran previous 2 lines sanity check passed 

In [43]:
# creating the feature matrix 
from sklearn.feature_extraction.text import TfidfVectorizer
import time

start_time = time.time()

acct_vectorizer1 = TfidfVectorizer(max_features=1000)
inj_vectorizer1 = TfidfVectorizer(max_features=1000)
acct_tfidf1 = acct_vectorizer1.fit_transform(df['Acc_Txt_Clned']).toarray()
inj_tfidf1 = inj_vectorizer1.fit_transform(df['Inj_Txt_Clned']).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 3.006840705871582 seconds ---


In [44]:
start_time = time.time()

acct_vectorizer2 = TfidfVectorizer(max_features=1000,
                                  ngram_range=(2, 2)) # look at two gram phrase only
inj_vectorizer2 = TfidfVectorizer(max_features=1000,
                                 ngram_range=(2, 2))
acct_tfidf2 = acct_vectorizer2.fit_transform(df['Acc_Txt_Clned']).toarray()
inj_tfidf2 = inj_vectorizer2.fit_transform(df['Inj_Txt_Clned']).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 4.046656608581543 seconds ---


In [45]:
acct_features1 = acct_vectorizer1.get_feature_names()
acct_features2 = acct_vectorizer2.get_feature_names()

inj_features1 = inj_vectorizer1.get_feature_names()
inj_features2 = inj_vectorizer2.get_feature_names()

### Create Additional Features

In [46]:
# add parking lot indicator and interaction
df['prklot_idx'] = np.where(acct_tfidf2[:,acct_features2.index('park lot')] > 0, 1, 0)
df['prklot_strain_sprain'] = np.where((df['prklot_idx'] == 1 & 
                                     ((df['Prim_NOI_Dsc'] == 'Strain') | 
                                      (df['Prim_NOI_Dsc'] == 'Sprain'))), 
                                    1,0)
df['prklot_head_category'] = np.where((df['prklot_idx'] == 1) & 
                                      (df['Prim_POB_Category_Dsc'] == 'HEAD') , 
                                      1,0)
df['prklot_knee'] = np.where((df['prklot_idx'] == 1) & 
                            (df['Prim_POB_Dsc'] == 'Knee') , 
                            1,0)

In [47]:
# add pallet indicator and interaction
df['pallet_idx'] = np.where((acct_tfidf1[:, acct_features1.index('pallet')] > 0) |
                             (inj_tfidf1[:, inj_features1.index('pallet')] > 0), 1, 0)
df['pallet_lowerEx'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Category_Dsc'] == 'LOWER EXTREMITIES') , 
                                1,0)
df['pallet_head_category'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Category_Dsc'] == 'HEAD') , 
                                1,0)
df['pallet_lowerbck'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Dsc'] == 'Low back (lumbar or lumbo-sacral area)') , 
                                1,0)
df['pallet_handarea'] = np.where((df['pallet_idx'] == 1) & 
                                (df['Prim_POB_Dsc'].isin(['Finger(s)', 'Wrist', 'Hand', 'Thumb'])) , 
                                1,0)

In [48]:
df['allrg_idx'] = np.where((acct_tfidf1[:,acct_features1.index('allergic')] > 0) |
                           (inj_tfidf1[:,inj_features1.index('allergic')] > 0), 1, 0)
df['notfeelwell_idx'] = np.where(acct_tfidf2[:,acct_features2.index('feel well')] > 0, 1, 0)

In [49]:
df['szre_stroke_idx'] = np.where(
                        (inj_tfidf1[:,inj_features1.index('strokeseizure')] > 0) |
                        (inj_tfidf1[:,inj_features1.index('seizure')] > 0) |
                        (inj_tfidf1[:,inj_features1.index('stroke')] > 0)
                        , 1, 0)

In [39]:
# export raw holdout modeldf 
#filename = 'WC_DeEscl_Holdout_Modeldf_v2.csv'
#df.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   #index=False)
import numpy as np
import pandas as pd

df = pd.read_csv('WC_DeEscl_Holdout_Modeldf_v2_16k.csv')

In [40]:
print('Data shape: ', df.shape)
df.head(10)

Data shape:  (16181, 114)


Year  Claim_ID  Claim_Num Claim_System_Entry_Dte  Report_dte  Injury_Dte  \
0  2013   2569906  12CA05378              3/13/2013   3/12/2013   4/23/2012   
1  2014   2817906  12CA06802             12/16/2014  12/16/2014   3/10/2014   
2  2015   2827706  949C55528              1/16/2015   1/16/2015  12/10/2014   
3  2015   2934006  12CA07022              6/24/2015   6/23/2015   1/23/2013   
4  2017   3371906  12CA08355             10/20/2017  10/18/2017    5/1/2015   
5  2013   5390730  905A25016              1/10/2013    1/9/2013    1/7/2013   
6  2013   5398230  905A25119              1/15/2013   1/14/2013    4/5/2010   
7  2013   5440930  197A46071               2/5/2013    2/4/2013    2/4/2013   
8  2013   5489330  116A30030               3/1/2013   2/28/2013   2/27/2013   
9  2013   5523430  868A62003              3/19/2013   3/18/2013   3/17/2013   

  Claim_Status_Cde Current_PMT_Cde Current_Controverted_I  \
0                F               3                    NaN   
1                F               3                    NaN   
2                F             NaN                    NaN   
3                O               3                    NaN   
4                F               3                    NaN   
5                F              0P                    NaN   
6                F               4                    NaN   
7                F               1                    NaN   
8                F              U6                    NaN   
9                F               4                    NaN   

   current_total_incurred  ...  prklot_head_category  prklot_knee  pallet_idx  \
0                       0  ...                     0            0           0   
1                       0  ...                     0            0           0   
2                       0  ...                     0            0           0   
3                       0  ...                     0            0           0   
4                       0  ...                     0            0           0   
5                     138  ...                     0            0           0   
6                       0  ...                     0            0           0   
7                     794  ...                     0            0           0   
8                    1793  ...                     0            0           0   
9                     805  ...                     0            0           0   

  pallet_lowerEx pallet_head_category  pallet_lowerbck pallet_handarea  \
0              0                    0                0               0   
1              0                    0                0               0   
2              0                    0                0               0   
3              0                    0                0               0   
4              0                    0                0               0   
5              0                    0                0               0   
6              0                    0                0               0   
7              0                    0                0               0   
8              0                    0                0               0   
9              0                    0                0               0   

  allrg_idx  notfeelwell_idx szre_stroke_idx  
0         0                0               0  
1         0                0               0  
2         0                0               0  
3         0                0               0  
4         0                0               0  
5         0                0               0  
6         0                0               0  
7         0                0               0  
8         0                0               0  
9         0                0               0  

[10 rows x 114 columns]

In [41]:
list(df.columns.values)

['Year',
 'Claim_ID',
 'Claim_Num',
 'Claim_System_Entry_Dte',
 'Report_dte',
 'Injury_Dte',
 'Claim_Status_Cde',
 'Current_PMT_Cde',
 'Current_Controverted_I',
 'current_total_incurred',
 'Indemnity_Incurred',
 'Medical_Incurred',
 'Expense_Incurred',
 'Account_Name',
 'Top Account Name',
 'Lost_Time_Cde',
 'Jurisdiction_State_Cde',
 'Intake_Method',
 'Primary_Nature_Of_Injury_Cde',
 'Secondary_Nature_Of_Injury_Cde',
 'Primary_Part_Of_Body_Cde1',
 'Secondary_Part_Of_Body_Cde',
 'Initial_Trtm_Cde',
 'Occurrence_Cde',
 'Claimant_Age_Qty',
 'Gender_Cde',
 'Tenure',
 'Report_Lag',
 'Avg_Wkly_Wge_Amt',
 'Runoff/Not Runoff',
 'Cust_Postal_Cde',
 'Claimant_Postal_Cde',
 'Occupation_Txt',
 'Accident_Note_Text',
 'Injury_Note_Text',
 'NAICS_Cde',
 'Segment_Today1',
 'Dataset1',
 'CRC_Rating_Plan_Code',
 'Initial_Seg',
 'InjurytoReport',
 'ReporttoEntry',
 'Prim_POB_Category_Cde',
 'Prim_POB_Dsc',
 'Prim_POB_Category_Dsc',
 'Scd_POB_Category_Cde',
 'Scd_POB_Dsc',
 'Scd_POB_Category_Dsc',
 'Occu

### Porcess and Encode Holdout Modeldf

In [42]:
target_col = [
             'current_total_incurred',
             'Indemnity_Incurred',
             'Medical_Incurred',
             'Expense_Incurred',    
             'deEscInd3500',
             'deEscInd3000',
             'deEscInd2500',
             'deEscInd2000',
             'deEscInd1500',
             'deEscInd1000',
             'over100kFlg',
             'zeroLossFlg']

analysis_col = [
             'Current_PMT_Cde',
             'Current_Controverted_I',
             'CRC_Rating_Plan_Code']

watch_col = [
            'Account_Name',
            'Top Account Name',
            'Account_Grp2',
            'Cust_Postal_Cde',
            'Claimant_Postal_Cde',
            'Occupation_Txt',
            'Occupation_Grp',
            'Occupation_Grp2',
            'NAICS_Cde',
            'PrimNOI_Grp20',
            'ScdNOI_Grp10',
            'PrimPOB_Grp30',
            'ScdPOB_Grp10',
            'Occur_Grp30']

other_col = [ 
             'Acc_Txt_Clned',
             'Inj_Txt_Clned',
             'Claim_ID',
             'Claim_Num',
             'Claim_System_Entry_Dte',
             'Report_dte',
             'Injury_Dte',
             'Claim_Status_Cde',
             'Jurisdiction_State_Cde',
             'Intake_Method',
             'Primary_Nature_Of_Injury_Cde',
             'Secondary_Nature_Of_Injury_Cde',
             'Primary_Part_Of_Body_Cde',
             'Secondary_Part_Of_Body_Cde',
             'Occurrence_Cde',
             'Report_Lag',
             'Runoff/Not Runoff',
             'Accident_Note_Text',
             'Injury_Note_Text',
             'Segment_Today',
             'Dataset',
             'Prim_POB_Category_Cde',
             'Scd_POB_Category_Cde',
             'Region',
             'Claimant_Latitude',
             'Claimant_Longitude',
             'Cust_Latitude',
             'Cust_Longitude',
             'Initial_Seg']

model_col = [
             'Year',
             'Initial_Trtm_Cde',
             'Lost_Time_Cde',
             'Claimant_Age_Qty',
             'Gender_Cde',
             'Tenure',
             'Avg_Wkly_Wge_Amt',
             'InjurytoReport',
             'ReporttoEntry',

             'Prim_POB_Dsc',
             'Prim_POB_Category_Dsc',
             'Scd_POB_Dsc',
             'Scd_POB_Category_Dsc',
             'Occur_Dsc',
             'Prim_NOI_Dsc',
             'Scd_NOI_Dsc',
 
             'Dist_to_work',   
             'age_5grp',
             'age_8grp',
             'age_10grp',
             'Tenure_5grp',
             'Avg_Wkly_Wge_Amt_5grp',
             'InjurytoReport_5grp',
             'ReporttoEntry_5grp',

             'Occupation_Grp1',
             'Account_Grp1',
             'NAICS_Grp',
             'CRC_Rating_Plan_Code',
        
             'PrimPOB_km_2c',
             'PrimPOB_km_4c',
             'PrimPOB_pam_2c',
             'PrimPOB_pam_5c',
             'PrimNOI_km_3c',
             'PrimNOI_km_6c',
             'PrimNOI_km_12c',
             'PrimNOI_pam_6c',
             'PrimNOI_pam_7c',
             'PrimNOI_pam_9c',
             'Occur_km_3c',
             'Occur_km_4c',
             'Occur_km_6c',
             'Occur_pam_4c',
             'Occur_pam_9c',
             'prklot_idx',
             'prklot_strain_sprain',
             'prklot_head_category',
             'prklot_knee',
             'pallet_idx',
             'pallet_lowerEx',
             'pallet_head_category',
             'pallet_lowerbck',
             'pallet_handarea',
             'allrg_idx',
             'notfeelwell_idx',
             'szre_stroke_idx']

In [43]:
moddf = df[model_col]
moddf.shape

(16181, 55)

In [44]:
moddf.dtypes

Year                       int64
Initial_Trtm_Cde          object
Lost_Time_Cde            float64
Claimant_Age_Qty           int64
Gender_Cde                object
Tenure                     int64
Avg_Wkly_Wge_Amt         float64
InjurytoReport             int64
ReporttoEntry              int64
Prim_POB_Dsc              object
Prim_POB_Category_Dsc     object
Scd_POB_Dsc               object
Scd_POB_Category_Dsc      object
Occur_Dsc                 object
Prim_NOI_Dsc              object
Scd_NOI_Dsc               object
Dist_to_work             float64
age_5grp                  object
age_8grp                  object
age_10grp                 object
Tenure_5grp               object
Avg_Wkly_Wge_Amt_5grp     object
InjurytoReport_5grp       object
ReporttoEntry_5grp        object
Occupation_Grp1           object
Account_Grp1              object
NAICS_Grp                 object
CRC_Rating_Plan_Code      object
PrimPOB_km_2c             object
PrimPOB_km_4c             object
PrimPOB_pa

In [45]:
moddf[['PrimNOI_km_3c',             
        'PrimNOI_km_6c',
        'PrimNOI_km_12c',       
        'PrimNOI_pam_6c',       
        'PrimNOI_pam_7c',       
        'PrimNOI_pam_9c',
        'PrimPOB_km_2c',
        'PrimPOB_km_4c',
        'PrimPOB_pam_2c',
        'PrimPOB_pam_5c']] = moddf[['PrimNOI_km_3c',             
                                    'PrimNOI_km_6c',
                                    'PrimNOI_km_12c',       
                                    'PrimNOI_pam_6c',       
                                    'PrimNOI_pam_7c',       
                                    'PrimNOI_pam_9c',
                                   'PrimPOB_km_2c',
                                    'PrimPOB_km_4c',
                                    'PrimPOB_pam_2c',
                                    'PrimPOB_pam_5c']].astype(str)

In [46]:
moddf.isnull().sum()

Year                        0
Initial_Trtm_Cde            0
Lost_Time_Cde            9329
Claimant_Age_Qty            0
Gender_Cde                  0
Tenure                      0
Avg_Wkly_Wge_Amt            0
InjurytoReport              0
ReporttoEntry               0
Prim_POB_Dsc                0
Prim_POB_Category_Dsc       0
Scd_POB_Dsc                 0
Scd_POB_Category_Dsc        0
Occur_Dsc                   0
Prim_NOI_Dsc                0
Scd_NOI_Dsc                 0
Dist_to_work             1723
age_5grp                    0
age_8grp                    0
age_10grp                   0
Tenure_5grp                 0
Avg_Wkly_Wge_Amt_5grp       0
InjurytoReport_5grp         0
ReporttoEntry_5grp          0
Occupation_Grp1             0
Account_Grp1                0
NAICS_Grp                   0
CRC_Rating_Plan_Code        0
PrimPOB_km_2c               0
PrimPOB_km_4c               0
PrimPOB_pam_2c              0
PrimPOB_pam_5c              0
PrimNOI_km_3c               0
PrimNOI_km

In [47]:
#fill na with 'missing'
moddf['Dist_to_work_missing'] = np.where(moddf['Dist_to_work'].isnull(), 1, 0)
moddf['Dist_to_work']  = moddf['Dist_to_work'].fillna(0.0)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
# sanity check
moddf[['Dist_to_work_missing']].groupby(['Dist_to_work_missing']).size()

Dist_to_work_missing
0    14458
1     1723
dtype: int64

In [49]:
adjcol = moddf.columns.values[np.where(moddf.isnull().sum() > 0)]
print(adjcol)
for col in adjcol:
    moddf[col] = moddf[col].fillna('missing')

['Lost_Time_Cde']


C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [50]:
# sanity check
moddf.isnull().sum()

Year                     0
Initial_Trtm_Cde         0
Lost_Time_Cde            0
Claimant_Age_Qty         0
Gender_Cde               0
Tenure                   0
Avg_Wkly_Wge_Amt         0
InjurytoReport           0
ReporttoEntry            0
Prim_POB_Dsc             0
Prim_POB_Category_Dsc    0
Scd_POB_Dsc              0
Scd_POB_Category_Dsc     0
Occur_Dsc                0
Prim_NOI_Dsc             0
Scd_NOI_Dsc              0
Dist_to_work             0
age_5grp                 0
age_8grp                 0
age_10grp                0
Tenure_5grp              0
Avg_Wkly_Wge_Amt_5grp    0
InjurytoReport_5grp      0
ReporttoEntry_5grp       0
Occupation_Grp1          0
Account_Grp1             0
NAICS_Grp                0
CRC_Rating_Plan_Code     0
PrimPOB_km_2c            0
PrimPOB_km_4c            0
PrimPOB_pam_2c           0
PrimPOB_pam_5c           0
PrimNOI_km_3c            0
PrimNOI_km_6c            0
PrimNOI_km_12c           0
PrimNOI_pam_6c           0
PrimNOI_pam_7c           0
P

In [51]:
# one-hot encoding categorical variable
import time
start_time = time.time()

X_holdout = moddf.copy()
catcol = X_holdout.select_dtypes(include=[object]).columns.values

X_holdout_onehot = X_holdout.copy()

for col in catcol:
    X_holdout_onehot = pd.get_dummies(X_holdout_onehot, 
                                    columns=[col], 
                                    prefix = [col])

print("--- %s seconds ---" % (time.time() - start_time))  

--- 2.3768362998962402 seconds ---


In [52]:
print(X_holdout_onehot.shape)

(16181, 534)


In [53]:
#X_holdout_onehot = dholdout

In [54]:
dropcol = ['PrimNOI_pam_9c_2',
            'PrimNOI_pam_6c_2',
            'Prim_NOI_Dsc_Byssinosis (lung disease from breathing cotton, flax or hemp)',
            'PrimNOI_km_6c_6',
            'Scd_NOI_Dsc_Silicosis (lung disease from breathing silica dust)',
            'PrimNOI_km_12c_2',
            'PrimNOI_pam_7c_2',
            'Prim_NOI_Dsc_Video display terminal related disease',
            'Scd_NOI_Dsc_Radiation',
            'Scd_NOI_Dsc_Asbestosis (lung disease from breathing asbestos)']

for col in dropcol:
    if col in list(X_holdout_onehot):
        print(col, X_holdout_onehot[col].sum())

In [55]:
#X_holdout_onehot = X_holdout_onehot.drop(['Scd_NOI_Dsc_Silicosis (lung disease from breathing silica dust)',
                                          #'Scd_NOI_Dsc_Asbestosis (lung disease from breathing asbestos)'], axis = 1)

print(X_holdout_onehot.shape)
list(X_holdout_onehot.columns)

(16181, 534)


['Year',
 'Claimant_Age_Qty',
 'Tenure',
 'Avg_Wkly_Wge_Amt',
 'InjurytoReport',
 'ReporttoEntry',
 'Dist_to_work',
 'prklot_idx',
 'prklot_strain_sprain',
 'prklot_head_category',
 'prklot_knee',
 'pallet_idx',
 'pallet_lowerEx',
 'pallet_head_category',
 'pallet_lowerbck',
 'pallet_handarea',
 'allrg_idx',
 'notfeelwell_idx',
 'szre_stroke_idx',
 'Dist_to_work_missing',
 'Initial_Trtm_Cde_0',
 'Initial_Trtm_Cde_1',
 'Initial_Trtm_Cde_2',
 'Initial_Trtm_Cde_3',
 'Initial_Trtm_Cde_4',
 'Initial_Trtm_Cde_5',
 'Initial_Trtm_Cde_missing',
 'Lost_Time_Cde_1.0',
 'Lost_Time_Cde_2.0',
 'Lost_Time_Cde_3.0',
 'Lost_Time_Cde_4.0',
 'Lost_Time_Cde_missing',
 'Gender_Cde_F',
 'Gender_Cde_M',
 'Prim_POB_Dsc_Abdomen',
 'Prim_POB_Dsc_Ankle',
 'Prim_POB_Dsc_Artificial Appliance',
 'Prim_POB_Dsc_Body Systems and Multiple Body Systems',
 'Prim_POB_Dsc_Brain',
 'Prim_POB_Dsc_Buttocks',
 'Prim_POB_Dsc_Chest, ribs or sternum',
 'Prim_POB_Dsc_Disc in low or middle back',
 'Prim_POB_Dsc_Disc in neck',
 'Pri

In [56]:
dtrain = pd.read_csv('WC_DeEscl_Data_Build_wClstr_v2_dtrain.csv')

#compare with the train dimension
set(list(dtrain)) - set(list(X_holdout_onehot))

{'Account_Grp1_ADP TOTALSOURCE GROUP, INC.',
 'Account_Grp1_ALLSTATE INSURANCE COMPANY',
 'Account_Grp1_AMERIGAS PARTNERS, L.P.',
 'Account_Grp1_AUTOMATIC DATA PROCESSING,',
 'Account_Grp1_BRE/EVERBRIGHT M6 LLC',
 'Account_Grp1_BRINKER INTERNATIONAL, INC.',
 'Account_Grp1_CARROLS CORPORATION',
 'Account_Grp1_COMCAST CORPORATION',
 'Account_Grp1_COSTCO WHOLESALE CORPORATION',
 'Account_Grp1_DARDEN RESTAURANTS, INC.',
 'Account_Grp1_DEAN FOODS COMPANY',
 'Account_Grp1_DOVER CORPORATION',
 'Account_Grp1_EMERSON ELECTRIC CO.',
 'Account_Grp1_FAIRFAX COUNTY PUBLIC SCHOOLS',
 'Account_Grp1_GENERAL MILLS, INC.',
 'Account_Grp1_HALLMARK CARDS, INC.',
 'Account_Grp1_IKEA HOLDING US, INC.',
 'Account_Grp1_LIBERTY MUTUAL HOLDING',
 'Account_Grp1_LSG SKY CHEFS, INC.',
 'Account_Grp1_MOHAWK INDUSTRIES, INC.',
 'Account_Grp1_NBC UNIVERSAL THEME PARKS',
 'Account_Grp1_NBCUNIVERSAL, LLC',
 'Account_Grp1_PENN NATIONAL GAMING, INC.',
 'Account_Grp1_REGAL ENTERTAINMENT GROUP',
 'Account_Grp1_ROCHE HOLDIN

In [57]:
X_holdout_onehot['Prim_NOI_Dsc_Enucleation (surgical removal of a tumor, eye, etc.)'] = 0
X_holdout_onehot['Account_Grp1_ADP TOTALSOURCE GROUP, INC.'] = 0
X_holdout_onehot['Account_Grp1_ALLSTATE INSURANCE COMPANY'] = 0
X_holdout_onehot['Account_Grp1_AMERIGAS PARTNERS, L.P.'] = 0
X_holdout_onehot['Account_Grp1_AUTOMATIC DATA PROCESSING,'] = 0
X_holdout_onehot['Account_Grp1_BRE/EVERBRIGHT M6 LLC'] = 0
X_holdout_onehot['Account_Grp1_BRINKER INTERNATIONAL, INC.'] = 0
X_holdout_onehot['Account_Grp1_CARROLS CORPORATION'] = 0
X_holdout_onehot['Account_Grp1_COMCAST CORPORATION'] = 0
X_holdout_onehot['Account_Grp1_COSTCO WHOLESALE CORPORATION'] = 0     
X_holdout_onehot['Account_Grp1_DARDEN RESTAURANTS, INC.'] = 0
X_holdout_onehot['Account_Grp1_DEAN FOODS COMPANY'] = 0
X_holdout_onehot['Account_Grp1_DOVER CORPORATION'] = 0
X_holdout_onehot['Account_Grp1_EMERSON ELECTRIC CO.'] = 0
X_holdout_onehot['Account_Grp1_FAIRFAX COUNTY PUBLIC SCHOOLS'] = 0
X_holdout_onehot['Account_Grp1_GENERAL MILLS, INC.'] = 0
X_holdout_onehot['Account_Grp1_HALLMARK CARDS, INC.'] = 0
X_holdout_onehot['Account_Grp1_IKEA HOLDING US, INC.'] = 0
X_holdout_onehot['Account_Grp1_LIBERTY MUTUAL HOLDING'] = 0
X_holdout_onehot['Account_Grp1_LSG SKY CHEFS, INC.'] = 0
X_holdout_onehot['Account_Grp1_MOHAWK INDUSTRIES, INC.'] = 0
X_holdout_onehot['Account_Grp1_NBC UNIVERSAL THEME PARKS'] = 0
X_holdout_onehot['Account_Grp1_NBCUNIVERSAL, LLC'] = 0
X_holdout_onehot['Account_Grp1_PENN NATIONAL GAMING, INC.'] = 0
X_holdout_onehot['Account_Grp1_REGAL ENTERTAINMENT GROUP'] = 0
X_holdout_onehot['Account_Grp1_ROCHE HOLDINGS, INC.'] = 0
X_holdout_onehot['Account_Grp1_RUAN TRANSPORT CORPORATION'] = 0
X_holdout_onehot['Account_Grp1_TD BANK, N.A.'] = 0
X_holdout_onehot['Account_Grp1_THE BON-TON STORES, INC.'] = 0
X_holdout_onehot['Account_Grp1_THE HOME DEPOT INC'] = 0
X_holdout_onehot['Account_Grp1_THE NEW JERSEY TRANSIT'] = 0
X_holdout_onehot['Account_Grp1_THE SHERWIN WILLIAMS COMPANY'] = 0
X_holdout_onehot['Account_Grp1_TRIDENT SEAFOODS CORPORATION'] = 0
X_holdout_onehot['Account_Grp1_TRIUMPH GROUP, INC.'] = 0
X_holdout_onehot['Account_Grp1_U.S. SECURITY ASSOCIATES'] = 0
X_holdout_onehot['Account_Grp1_WAWA, INC.'] = 0
X_holdout_onehot['Account_Grp1_missing'] = 0
X_holdout_onehot['CRC_Rating_Plan_Code_missing'] = 0
X_holdout_onehot['Occupation_Grp1_ASSET PROTECTI'] = 0
X_holdout_onehot['Occupation_Grp1_ASSISTANTASSO'] = 0
X_holdout_onehot['Occupation_Grp1_BUILD OUT CREW'] = 0
X_holdout_onehot['Occupation_Grp1_FABRICATORCOU'] = 0
X_holdout_onehot['Occupation_Grp1_MERCHANDISING'] = 0
X_holdout_onehot['Occupation_Grp1_NE PREORD DEL'] = 0
X_holdout_onehot['Occupation_Grp1_SERVICE DELI'] = 0
X_holdout_onehot['Occupation_Grp1_SERVICECONCIE'] = 0
X_holdout_onehot['Occupation_Grp1_SUP HOURLY'] = 0
X_holdout_onehot['Occupation_Grp1_TOOL RENTAL AS'] = 0
X_holdout_onehot['Occupation_Grp1_TRAINING COORD'] = 0
X_holdout_onehot['Occur_Dsc_Crash of airplane'] = 0
X_holdout_onehot['Occur_Dsc_Crash of rail vehicle'] = 0
X_holdout_onehot['Occur_Dsc_burn or scald from abnormal air pressure'] = 0
#X_holdout_onehot['Occur_Dsc_burn or scald from cold objects or substances'] = 0
X_holdout_onehot['Occur_Dsc_natural disasters'] = 0
X_holdout_onehot['Occur_Dsc_terrorism'] = 0
X_holdout_onehot['Occur_km_3c_Crash of airplane'] = 0
X_holdout_onehot['Occur_km_4c_Crash of airplane'] = 0
X_holdout_onehot['Occur_km_6c_Crash of airplane'] = 0
X_holdout_onehot['Occur_pam_4c_Crash of airplane'] = 0
X_holdout_onehot['Occur_pam_9c_Crash of airplane'] = 0
X_holdout_onehot['PrimNOI_km_12c_1'] = 0
X_holdout_onehot['PrimNOI_km_6c_3'] = 0
X_holdout_onehot['PrimNOI_pam_6c_3'] = 0
X_holdout_onehot['PrimNOI_pam_7c_3'] = 0
X_holdout_onehot['PrimNOI_pam_9c_3'] = 0
#X_holdout_onehot['PrimPOB_km_2c_Disc in neck'] = 0
#X_holdout_onehot['PrimPOB_km_4c_Disc in neck'] = 0
#X_holdout_onehot['PrimPOB_pam_2c_Disc in neck'] = 0
#X_holdout_onehot['PrimPOB_pam_5c_Disc in neck'] = 0
X_holdout_onehot['Prim_NOI_Dsc_Aids'] = 0
X_holdout_onehot['Prim_NOI_Dsc_Black Lung disease'] = 0
#X_holdout_onehot['Prim_NOI_Dsc_Enucleation (surgical removal of a tumor, eye, etc.)'] = 0
X_holdout_onehot['Prim_NOI_Dsc_Mental disorder'] = 0
X_holdout_onehot['Prim_NOI_Dsc_Silicosis (lung disease from breathing silica dust)'] = 0
#X_holdout_onehot['Prim_POB_Dsc_Disc in neck'] = 0
X_holdout_onehot['Prim_POB_Dsc_Spinal cord (neck area)'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Amputation'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Asphyxiation'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Cancer'] = 0
#X_holdout_onehot['Scd_NOI_Dsc_Electric Shock'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Enucleation (surgical removal of a tumor, eye, etc.)'] = 0
#X_holdout_onehot['Scd_NOI_Dsc_Freezing'] = 0
X_holdout_onehot['Scd_NOI_Dsc_General Poisoning'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Heat prostration (heat stroke)'] = 0
#X_holdout_onehot['Scd_NOI_Dsc_Hernia'] = 0
#X_holdout_onehot['Scd_NOI_Dsc_Loss of hearing'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Metal poisoning'] = 0
#X_holdout_onehot['Scd_NOI_Dsc_Multiple Injuries'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Myocardial infarction (heart attack)'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Occupational disease'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Severence'] = 0
X_holdout_onehot['Scd_NOI_Dsc_Vision loss'] = 0
X_holdout_onehot['Scd_POB_Dsc_Disc in neck'] = 0
X_holdout_onehot['Scd_POB_Dsc_Skull'] = 0           
X_holdout_onehot['Scd_POB_Dsc_Spinal cord (neck area)'] = 0     
X_holdout_onehot['deEscInd3000'] = df['deEscInd3000']

In [58]:
# export encoded holdout modeling data
X_holdout_onehot.to_csv('WC_DeEscl_Data_Holdout_wClstr_v2_dholdout.csv', index=False)